In [19]:
from fishsense_api_sdk.client import Client
from label_studio_sdk.client import LabelStudio
from pathlib import Path
import cv2
from tqdm.notebook import tqdm

In [2]:
URL = "http://localhost:8000"
DIVE_ID = 279
PROJECT_ID = 57

In [14]:
OUTPUT_FOLDER = (Path("../output") / "preprocess_groups_jpeg").absolute()

In [3]:
label_studio_api_key = Path("..") / ".label_studio_api_key"

label_studio_api_key.exists()

True

In [4]:
fs = Client(URL)
ls = LabelStudio(base_url=f"https://labeler.e4e.ucsd.edu", api_key=label_studio_api_key.read_text().strip())

In [5]:
dive = await fs.dives.get(dive_id=DIVE_ID)

dive

Dive(id=279, name='111323_Alligator DeepEast Drift2_FSL05', path='drive-download-20240307T1050Z/112023_Alligator/111323_Alligator/111323_Alligator_FSL05/111323_Alligator DeepEast Drift2_FSL05', dive_datetime=datetime.datetime(2023, 11, 13, 12, 34, 8, tzinfo=TzInfo(0)), priority=<Priority.HIGH: 'HIGH'>, flip_dive_slate=None, camera_id=5, dive_slate_id=None)

In [6]:
images = await fs.images.get(dive_id=dive.id)

len(images), images[0]

(155,
 Image(id=78666, path='drive-download-20240307T1050Z/112023_Alligator/111323_Alligator/111323_Alligator_FSL05/111323_Alligator DeepEast Drift2_FSL05/PB130168.ORF', taken_datetime=datetime.datetime(2023, 11, 13, 12, 3, 31, tzinfo=TzInfo(0)), checksum='7252dfb0db0b855616e45607f58c1c93', is_canonical=True, dive_id=279, camera_id=5))

In [ ]:
tasks = []

for image in tqdm(images):
    laser_label = await fs.labels.get_laser_label(image_id=image.id)
    image_path = OUTPUT_FOLDER / f"{image.checksum}.JPG"

    img = cv2.imread(str(image_path))
    height, width, _ = img.shape

    if laser_label is None or laser_label.label is None:
        predictions = []
    else:
        predictions = [{
            "model_version": "sql",
            "result": [{
                "id": "result1",
                "type": "keypointlabels",        
                "to_name": "image", "from_name": "laser",
                "original_width": width, "original_height": height,
                "image_rotation": 0,
                "value": {
                    "x": laser_label.x / width * 100,
                    "y": laser_label.y / height * 100,
                    "width": 0.2,
                    "keypointlabels": [laser_label.label]
                }
            }]
        }]

    tasks.append({
        "data": {
            "image": f"https://orchestrator.fishsense.e4e.ucsd.edu/api/v1/data/groups_jpeg/{image.checksum}"
        },
        "predictions": predictions,
        "annotations": []
    })

  0%|          | 0/155 [00:00<?, ?it/s]

In [21]:
imported_tasks = ls.projects.import_tasks(PROJECT_ID, request=tasks, return_task_ids=True)

imported_tasks

ProjectsImportTasksResponse(annotation_count=0, could_be_tasks_list=False, data_columns=[], duration=0.38053393363952637, file_upload_ids=[], found_formats=[], predictions_count=None, task_count=155, prediction_count=117, task_ids=[222252, 222253, 222254, 222255, 222256, 222257, 222258, 222259, 222260, 222261, 222262, 222263, 222264, 222265, 222266, 222267, 222268, 222269, 222270, 222271, 222272, 222273, 222274, 222275, 222276, 222277, 222278, 222279, 222280, 222281, 222282, 222283, 222284, 222285, 222286, 222287, 222288, 222289, 222290, 222291, 222292, 222293, 222294, 222295, 222296, 222297, 222298, 222299, 222300, 222301, 222302, 222303, 222304, 222305, 222306, 222307, 222308, 222309, 222310, 222311, 222312, 222313, 222314, 222315, 222316, 222317, 222318, 222319, 222320, 222321, 222322, 222323, 222324, 222325, 222326, 222327, 222328, 222329, 222330, 222331, 222332, 222333, 222334, 222335, 222336, 222337, 222338, 222339, 222340, 222341, 222342, 222343, 222344, 222345, 222346, 222347, 